In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Function to extract MFCCs from an audio file
def extract_features(audio_path, n_mfcc=13):
    audio, sr = librosa.load(audio_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfccs, axis=1)

# Load and preprocess the dataset
def load_data(data_dir):
    features = []
    labels = []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for audio_file in os.listdir(label_dir):
                file_path = os.path.join(label_dir, audio_file)
                mfccs = extract_features(file_path)
                features.append(mfccs)
                labels.append(label)
    return np.array(features), np.array(labels)

# Load and preprocess the data
X, y = load_data("/content/drive/MyDrive/IBM_audio_dataset")
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Normalize the features
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean) / (std + 1e-8)
X_test = (X_test - mean) / (std + 1e-8)

# Reshape for CNN input (if necessary)
if len(X_train.shape) == 3:
    X_train = X_train[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of X: (8022, 13)
Shape of y: (8022,)
Shape of X_train: (6417, 13)
Shape of X_test: (1605, 13)


## Logistic Regression and Random Forest

In [ ]:
def train_logistic_regression(X_train, y_train, X_test, y_test):
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(X_train.reshape(X_train.shape[0], -1), y_train)  # Flatten the input
    y_pred = log_reg.predict(X_test.reshape(X_test.shape[0], -1))
    print("\nLogistic Regression:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))

def train_random_forest(X_train, y_train, X_test, y_test):
    rf = RandomForestClassifier(n_estimators=100)
    rf.fit(X_train.reshape(X_train.shape[0], -1), y_train)
    y_pred = rf.predict(X_test.reshape(X_test.shape[0], -1))
    print("\nRandom Forest:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))

train_logistic_regression(X_train, y_train, X_test, y_test)
train_random_forest(X_train, y_train, X_test, y_test)


Logistic Regression:
Accuracy: 0.729595015576324
              precision    recall  f1-score   support

           0       0.70      0.82      0.75       332
           1       0.50      0.31      0.38       295
           2       0.80      0.86      0.83       885
           3       0.61      0.52      0.56        93

    accuracy                           0.73      1605
   macro avg       0.65      0.63      0.63      1605
weighted avg       0.71      0.73      0.71      1605


Random Forest:
Accuracy: 0.9557632398753894
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       332
           1       0.96      0.93      0.94       295
           2       0.97      0.98      0.97       885
           3       0.96      0.84      0.90        93

    accuracy                           0.96      1605
   macro avg       0.95      0.93      0.94      1605
weighted avg       0.96      0.96      0.96      1605

